# Practice Exercises for Single Table SQL

Given a dataset and a set of questions, form the queries to answer the questions. All questions in this particular exercise only require a query to a single table. 

⚠️ *Note:* Please display minimal number of attributes (columns) needed to make sense of your answer. Not too many, not too few.


## Setup
The SQLite database for these exercises has been setup for you. For the curious, you can look at this file for the DDL commands to setup the tables and insert data.  `db_illum_ddl_5_3.sql`

Here is the schema for the database. The arrows show foreign keys, which is just fyi for now. 

![worker_project schema](worker_project_schema.png) 

In [2]:
# Make sure to always run this cell when you open this notebook
# Start the Jupyter SQL engine, connecting to a SQLite database 
%reload_ext sql 
%sql sqlite:///worker_project.db

0. In the command line, inspect the sqlite database (worker_project.db) to see what the tables and columns actually are. Any differences you observe? (Also, copy past your output in the cell below)

In [ ]:
# Tables: Assign   Dept     Project  Worker
# Columns: 
# Assign - projNo|empId|hoursAssigned|rating
# Dept -  deptName|mgrId
# Project - projNo|projName|projMgrId|budget|startDate|expectedDurationWeeks
# Worker - empId|lastName|firstName|deptName|birthDate|hireDate|salary

# Worker table has lastName and firstName instead of empName, and hireDate instead of dateHired.
# Project has expectedDurationWeeks instead of expectedDuration.

1. Get the names of all workers in the Accounting department.

In [2]:
%%sql
select firstName, lastName from worker
where deptName = "Accounting";

 * sqlite:///worker_project.db
Done.


firstName,lastName
Tom,Smith
Mary,Jones
Jane,Burns


2. Get the name of the employee in the Research department who has the lowest salary.

In [6]:
%%sql
select firstName, lastName, min(salary) from worker
where deptName = "Research";

 * sqlite:///worker_project.db
Done.


firstName,lastName,min(salary)
Amanda,Chin,60000


3. Get the details of the project with the highest budget.

In [7]:
%%sql
select * from project
order by budget desc

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1025,Neptune,110,600000,01-Feb-2021,45
1005,Saturn,101,400000,01-Jun-2022,35
1030,Pluto,110,380000,15-Sep-2023,50
1019,Mercury,110,350000,15-Feb-2022,40
1001,Jupiter,101,300000,01-Feb-2021,50


## The following questions may require more than one query to answer. 
Show your work! Add as many SQL cells as needed for each question. Use comments to indicate how the output of each query is used to arrive at your answer.  

4. List out alphabetically the names of all workers working on Project 1001.

In [8]:
%%sql
-- use empId to find out which employees work on project 1001
select projNo, empId from assign
where projNo = "1001"

 * sqlite:///worker_project.db
Done.


projNo,empId
1001,101
1001,103
1001,105
1001,115


In [15]:
%%sql
-- use empId results from previous query
-- list in alphabetical order by last name
select empId, lastName, firstName from worker
where empId = 101 or empId = 103 or empId = 105 or empId = 115
order by lastName asc;

 * sqlite:///worker_project.db
Done.


empId,lastName,firstName
105,Burns,Jane
115,Chin,Amanda
103,Jones,Mary
101,Smith,Tom


5. Get an alphabetical list of names and corresponding ratings for all workers on any project managed by Michael Burns.

In [6]:
%%sql
-- determine Michael's empId to use for later queries
select empId, firstName, lastName from worker
where firstName = "Michael"

 * sqlite:///worker_project.db
Done.


empId,firstName,lastName
110,Michael,Burns


In [7]:
%%sql
-- get project info from projects that Michael has managed
-- using previous empId value for projMgrId
select projNo, projName, projMgrId from project
where projMgrId = 110

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId
1019,Mercury,110
1025,Neptune,110
1030,Pluto,110


In [15]:
%%sql
-- finds all employee IDs for employees working on projects that Michael managed
-- only one empId is different from projMgrId
select projNo, empId, rating from assign
where projNo = 1019 or projNo = 1025 or projNo = 1030

 * sqlite:///worker_project.db
Done.


projNo,empId,rating
1019,110,5
1019,115,4
1025,110,None
1030,110,None


In [17]:
%%sql
-- gets workers' names from worker table
-- gets project number and rating from assign table
select worker.empId, worker.firstName, worker.lastName, assign.projNo, assign.rating
from worker
join assign
on worker.empId = assign.empId
where projNo = 1019 or projNo = 1025 or projNo = 1030
order by worker.empId

 * sqlite:///worker_project.db
Done.


empId,firstName,lastName,projNo,rating
110,Michael,Burns,1019,5
110,Michael,Burns,1025,None
110,Michael,Burns,1030,None
115,Amanda,Chin,1019,4


6. For all projects starting after Feb 1, 2022, find the project number and names of all workers assigned to them. 

In [19]:
%%sql
-- gets project numbers within timeframe
select projNo, startDate from project
where startDate >= '01-Feb-2022'

 * sqlite:///worker_project.db
Done.


projNo,startDate
1005,01-Jun-2022
1019,15-Feb-2022
1030,15-Sep-2023


In [21]:
%%sql
-- gets IDs for employees who worked on the projects through assign table
-- gets worker names through worker table
select assign.projNo, assign.empId, worker.firstName, worker.lastName
from assign
join worker
on worker.empId = assign.empId
where projNo = 1005 or projNo = 1019 or projNo = 1030

 * sqlite:///worker_project.db
Done.


projNo,empId,firstName,lastName
1005,103,Mary,Jones
1019,110,Michael,Burns
1019,115,Amanda,Chin
1030,110,Michael,Burns


7. Find the names of employees who are not assigned as managers any project.

In [22]:
%%sql
-- finds all employees who are project managers
select projMgrId from project

 * sqlite:///worker_project.db
Done.


projMgrId
101
101
110
110
110


In [33]:
%%sql
-- gets names of employees who weren't in previous query results
select lastName, firstName, empId from worker
where empId != 101 and empId != 110
order by lastName

 * sqlite:///worker_project.db
Done.


lastName,firstName,empId
Burns,Jane,105
Chin,Amanda,115
Jones,Mary,103


8. Find details of any project with the string "urn" in the project name. 

In [35]:
%%sql
-- * gets all details
-- %% used to show that other text can be on either side of the string value
select * from project
where projName like '%urn%'

 * sqlite:///worker_project.db
Done.


projNo,projName,projMgrId,budget,startDate,expectedDurationWeeks
1005,Saturn,101,400000,01-Jun-2022,35


9. List out the projects from most to least efficient. (hint: the project with the lowest budget is not necessarily the one that uses time and human resources effectively.)

In [42]:
%%sql
-- shortest projects come to the top as most efficient
-- when duration is equal, lower budget is more efficient
-- projName included to easily identify the project
select projName, budget, expectedDurationWeeks from project
order by expectedDurationWeeks asc, budget asc

 * sqlite:///worker_project.db
Done.


projName,budget,expectedDurationWeeks
Saturn,400000,35
Mercury,350000,40
Neptune,600000,45
Jupiter,300000,50
Pluto,380000,50


10. Explain your reasoning for your solution to #9 (use a Markdown cell), including who should get the award for Most Efficient Project Manager.

In #9, I used expectedDurationWeeks as the primary indicator for the most efficient project. When more than one project had the same duration, the one with the lower budget was used as a tie-breaker and considered to be more efficient. Time is the most important resource, so spending less time on a project is almost always more efficient. 

As for the Most Efficient Project Manager, I think that the manager over the Saturn project should receive the award. This project has the shortest expected duration, and the budget seems reasonable as well. 

11. Bonus!!  A Query that you can't easily answer with just a set of queries: Get a list of project numbers with name and start dates, of all the projects that have the same start date as another. 
*you don't have to answer this, but think about it!*


In [57]:
%%sql
-- not sure how to list more than one project that meets the criteria
select projNo, projName, startDate, count(startDate)
from project
group by startDate
having count(startDate) > 1

 * sqlite:///worker_project.db
Done.


projNo,projName,startDate,count(startDate)
1001,Jupiter,01-Feb-2021,2


## Submission:

Commit and push your changes to this document by the deadline, and share a link to the repository in the Moodle assignment.  